In [5]:
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist

(x_train,y_train), (x_test,y_test) = mnist.load_data()

x_train, x_test = x_train/255.0, x_test/255.0

import tensorflow_datasets as tfds
import tensorflow as tf

#print(tfds.list_builders())
dataset, info = tfds.load('fashion_mnist', as_supervised = True, with_info = True)
dataset_test, dataset_train = dataset['test'], dataset['train']
#print(info)
def convert_types(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

batch_size = 32

dataset_train = dataset_train.map(convert_types).shuffle(10000).batch(batch_size)
dataset_test = dataset_test.map(convert_types).batch(batch_size)

<class 'numpy.ndarray'>
<class 'tuple'>


AttributeError: module 'tensorflow_datasets' has no attribute 'load'

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print("Generator")
#datagen = ImageDataGenerator()
datagen = ImageDataGenerator(rotation_range = 10, horizontal_flip = True, zoom_range = 0.1)

Generator


In [9]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Flatten(input_shape=(28, 28)),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(10,activation='softmax')
# ])

from tensorflow.keras.layers import Conv2D, Flatten, Dense, BatchNormalization, Dropout, Activation, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras import Model

class CNNModel(Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        drop_rate = 0.5
        
        self._layers = ([
            Conv2D(32, 3), # 28, 28, 1 -> 26, 26, 32
            BatchNormalization(),
            Activation(tf.nn.relu),
            Conv2D(64, 3), # 26, 26, 32 -> 24, 24, 64
            BatchNormalization(),
            Activation(tf.nn.relu),
            MaxPool2D(), # 24, 24, 64 -> 12, 12, 64
            Conv2D(128, 3), # 12, 12, 64 -> 10, 10, 128
            BatchNormalization(),
            Activation(tf.nn.relu),
            Conv2D(256, 3), # 10, 10, 128 -> 8, 8, 256
            BatchNormalization(),
            Activation(tf.nn.relu),
            MaxPool2D(), # 8, 8, 256 -> 4, 4, 256
            Flatten(), # 4, 4, 256 -> 4096
            Dense(256), # 4096 -> 256
            BatchNormalization(),
            Dropout(drop_rate),
            Activation(tf.nn.relu),
            Dense(10, activation = 'softmax') # 256 -> 10                        
        ])                
        
    def call(self, x):
        for layer in self._layers:
            x = layer(x)
        return x
       
    
model = CNNModel()

In [10]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

In [11]:
@tf.function
def train_step(image, label):
    with tf.GradientTape() as tape:
        predictions = model(image)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label, predictions)
        
@tf.function
def test_step(image, label):
    predictions = model(image)
    loss = loss_object(label, predictions)
    
    test_loss(loss)
    test_accuracy(label, predictions)

In [ ]:
import time

num_epoch = 80
start_time = time.time()

#train_accuracies = []
#test_accuracies = []
train_accuracies_with_da = []
test_accuracies_with_da = []


for epoch in range(num_epoch):    
    for image, label in dataset_train:
        for _image, _label in datagen.flow(image, label, batch_size = batch_size):
            train_step(_image, _label)
            break
        
    for test_image, test_label in dataset_test:
        test_step(test_image, test_label)
        
    #train_accuracies.append(train_accuracy.result())
    #test_accuracies.append(test_accuracy.result())
    train_accuracies_with_da.append(train_accuracy.result())
    test_accuracies_with_da.append(test_accuracy.result())
    
    
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}, spent_time: {} min'
    spent_time = time.time() - start_time
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(), test_accuracy.result() * 100, spent_time / 60))
    

Epoch 1, Loss: 0.4940635859966278, Accuracy: 81.6066665649414, Test Loss: 0.3473312258720398, Test Accuracy: 87.44000244140625, spent_time: 2.556767725944519 min
Epoch 2, Loss: 0.4113214313983917, Accuracy: 84.74833679199219, Test Loss: 0.31387659907341003, Test Accuracy: 88.5, spent_time: 5.116659490267436 min
Epoch 3, Loss: 0.3695756196975708, Accuracy: 86.29500579833984, Test Loss: 0.3014501631259918, Test Accuracy: 89.10333251953125, spent_time: 7.647784769535065 min
Epoch 4, Loss: 0.3430013656616211, Accuracy: 87.30791473388672, Test Loss: 0.28941860795021057, Test Accuracy: 89.51499938964844, spent_time: 10.174489609400432 min
Epoch 5, Loss: 0.32330355048179626, Accuracy: 88.03199768066406, Test Loss: 0.2830665409564972, Test Accuracy: 89.78400421142578, spent_time: 12.694288484255473 min
Epoch 6, Loss: 0.3080766201019287, Accuracy: 88.58777618408203, Test Loss: 0.27736398577690125, Test Accuracy: 90.06000518798828, spent_time: 15.224726307392121 min
Epoch 7, Loss: 0.295229971408

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_accuracies, label = 'Train Accuracy')
plt.plot(test_accuracies, linestyle = 'dashed', label = 'Test Accuracy')
plt.plot(train_accuracies_with_da, label = 'Train Accuracy with Data Augmentation')
plt.plot(test_accuracies_with_da, linestyle = 'dashed', label = 'Test Accuracy with Data Augmentation')
plt.legend()
plt.show()